In [27]:

from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import wave
import numpy as np
from music21 import converter, instrument, note, chord
import glob
import os
import Paths
from pydub import AudioSegment
from keras.models import Sequential
from keras.layers import Dense, LSTM, MaxPooling1D, Dropout, Conv1D
import librosa
import tensorflow as tf
from keras.layers import LeakyReLU
from keras import models, layers
from StatefulModel import StatefulModel
import sys, time
import matplotlib.pyplot as plt

def define_model(len_ts,
                 hidden_neurons = 10,
                 nfeature=1,
                 batch_size=None,
                 stateful=False):
    in_out_neurons = 1

    inp = layers.Input(batch_shape= (batch_size, len_ts, nfeature),
                       name="input")

    rnn = layers.LSTM(hidden_neurons,
                    return_sequences=True,
                    stateful=stateful,
                    name="RNN")(inp)

    dens = layers.TimeDistributed(layers.Dense(in_out_neurons,name="dense"))(rnn)
    model = models.Model(inputs=[inp],outputs=[dens])

    model.compile(loss="mean_squared_error",
                  sample_weight_mode="temporal",
                  optimizer="rmsprop")
    return(model,(inp,rnn,dens))




directory = os.listdir(Paths.IN_DIRECTORY)
array = []

sequence_length = []
for sample_name in directory:
    if 'wav' in sample_name:
        file_path = os.path.join(Paths.IN_DIRECTORY, sample_name)
        song, sr = librosa.load(file_path)
        sequence_length.append(len(song))
        #print(len(song))
        #song = AudioSegment.from_wav(file_path)
        #samples = song.get_array_of_samples()
        array.append(song)

# Removing the last element using slicing
lastElementIndex = len(array)-1
in_array = array[:lastElementIndex]
out_array = array[1:]

# Get overall number of Input values from samples
#Number_of_values = sum(x for x in sequence_length)


# Create the Input and Output to tensors
input_tensor = tf.convert_to_tensor(in_array)
output_tensor = tf.convert_to_tensor(out_array)

batch_size = len(input_tensor)
timestep = sequence_length[0]

input_data = tf.reshape(input_tensor, shape=(batch_size, timestep, 1))
output_data = tf.reshape(output_tensor, shape=(batch_size, timestep, 1))

prop_train = 0.8
ntrain = int(input_data.shape[0]*prop_train)

D=100

w = np.zeros(output_data.shape[:2])
w[:,D:] = 1
w_train = w

X_train, X_val = input_data[:ntrain], input_data[ntrain:]
y_train, y_val = output_data[:ntrain], output_data[ntrain:]
w_train, w_val = w[:ntrain], w[ntrain:]

hunits = 64

model_stateful, _ = define_model(
    hidden_neurons = hunits,
    batch_size=400,
    stateful=True,
    len_ts = 500)

#model_stateful.summary()

hunits = 64
model_stateless, _ = define_model(
                    hidden_neurons = hunits,
                    len_ts = X_train.shape[1])
model_stateless.summary()

start = time.time()
history = model_stateless.fit(X_train,y_train,
                             batch_size=batch_size,
                             epochs=20,
                             verbose=1,
                              sample_weight=w_train,
                             validation_data=(X_val,y_val,w_val))
end = time.time()
print("Time Took :{:3.2f} min".format( (end-start)/60 ))

'''

smodel = StatefulModel(model=model_stateful, print_val_every=500)

start = time.time()
smodel.fit(X_train, y_train, w_train,
                              X_val, y_val, w_val,
                              Nepoch=100)

end = time.time()
print("Time Took {:3.2f} min".format((end - start) / 60))
'''

Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 66150, 1)]        0         
                                                                 
 RNN (LSTM)                  (None, 66150, 64)         16896     
                                                                 
 time_distributed_13 (TimeDi  (None, 66150, 1)         65        
 stributed)                                                      
                                                                 
Total params: 16,961
Trainable params: 16,961
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
1/1 [==============================] - 23s 23s/step - loss: 0.0207 - val_loss: 0.0595
Epoch 2/20
1/1 [==============================] - 24s 24s/step - loss: 0.0196 - val_loss: 0.0601
Epoch 3/20
1/1 [==============================] - 23s 23s/

'\n\nsmodel = StatefulModel(model=model_stateful, print_val_every=500)\n\nstart = time.time()\nsmodel.fit(X_train, y_train, w_train,\n                              X_val, y_val, w_val,\n                              Nepoch=100)\n\nend = time.time()\nprint("Time Took {:3.2f} min".format((end - start) / 60))\n'

In [31]:
# Evaluate the model on the test data using evaluate
print("Evaluate on test data")
results = model_stateless.evaluate(input_tensor, output_tensor, batch_size)
print("test loss, test acc:", results)

Evaluate on test data
1/1 [==============================] - 3s 3s/step - loss: 0.0281
test loss, test acc: 0.028130121529102325


In [30]:
X_train[1]

<tf.Tensor: shape=(66150, 1), dtype=float32, numpy=
array([[-0.19236994],
       [-0.29156643],
       [-0.25135237],
       ...,
       [-0.38337252],
       [-0.36457327],
       [-0.38309956]], dtype=float32)>

In [23]:
X_train[0]

<tf.Tensor: shape=(66150, 1), dtype=float32, numpy=
array([[ 0.        ],
       [ 0.        ],
       [ 0.        ],
       ...,
       [-0.2871928 ],
       [-0.27692395],
       [-0.29513258]], dtype=float32)>